In [3]:
import os,os.path as osp
from glob import glob
from tqdm import tqdm
import shutil
import random

import cv2
import numpy as np

from ultralytics import YOLO
from ultralytics import settings

import yaml
from sklearn.model_selection import train_test_split

settings.update({"wandb": False})

In [4]:
dataset_dir = '/kaggle/input/vehicle-detection/TrafficPublic'
train_dir = dataset_dir + '/train'
test_dir = dataset_dir + '/test'
save_dir = '/kaggle/working/yolov8'
val_ratio = 0.2

# Create folder dataset for yolov8
os.makedirs(save_dir,exist_ok=True)
os.makedirs(save_dir + '/' + 'images',exist_ok=True)
os.makedirs(save_dir + '/' + 'labels',exist_ok=True)

os.makedirs(save_dir + '/' + 'images/train',exist_ok=True)
os.makedirs(save_dir + '/' + 'labels/train',exist_ok=True)

os.makedirs(save_dir + '/' + 'images/val',exist_ok=True)
os.makedirs(save_dir + '/' + 'labels/val',exist_ok=True)

# List annotation file  
ann_paths = glob(osp.join(train_dir , '*.txt'))
ann_train, ann_val = train_test_split(ann_paths, test_size=val_ratio)

# Copy train images and labels folder
print('Copy images and labels in train folder')
for ann_path in tqdm(ann_train):
    filename = osp.split(ann_path[:-4])[-1]
    
    img_path = ann_path[0:-4] + '.jpg'
    save_img_path = save_dir + '/images/train/' + filename + '.jpg'
    save_label_path = save_dir + '/labels/train/' + filename + '.txt'
    
    if os.path.exists(img_path):  
        shutil.copy(img_path, save_img_path)
        shutil.copy(ann_path, save_label_path)

# Copy val images and labels folder
print('Copy images and labels in val folder')
for ann_path in tqdm(ann_val):
    filename = osp.split(ann_path[:-4])[-1]
    
    img_path = ann_path[0:-4] + '.jpg'
    save_img_path = save_dir + '/images/val/' + filename + '.jpg'
    save_label_path = save_dir + '/labels/val/' + filename + '.txt'
    
    if os.path.exists(img_path):  
        shutil.copy(img_path, save_img_path)
        shutil.copy(ann_path,  save_label_path)
    
# Create .yaml yolo format
print('Create config file dataset.yaml')
classes_list = []
index = 0

for label in open(train_dir + "/classes.txt", "r").read().split('\n'): 
    classes_list.append(str(index) + ': ' + label)
    index += 1
    
data = {
    "path" : save_dir,
    "train" : save_dir + '/' + 'images/train',
    "val" : save_dir + '/' + 'images/val',
    "names" : classes_list
}

with open('yolov8/dataset.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False, sort_keys=False)

Copy images and labels in train folder


100%|██████████| 1459/1459 [00:14<00:00, 102.72it/s]


Copy images and labels in val folder


100%|██████████| 365/365 [00:03<00:00, 107.54it/s]

Create config file dataset.yaml


In [17]:
model = YOLO('yolov8x.pt')  # load a pretrained model (recommended for training)
results = model.train(data='yolov8/dataset.yaml',project='detect',name = 'train', epochs=20)

Ultralytics YOLOv8.2.48 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=yolov8/dataset.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=detect, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

train: Scanning /kaggle/working/yolov8/labels/train.cache... 1458 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1458/1458 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolov8/labels/val.cache... 365 images, 0 backgrounds, 0 corrupt: 100%|██████████| 365/365 [00:00<?, ?it/s]


Plotting labels to detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to detect/train4
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20      15.5G      1.564      0.959      1.452         11        640: 100%|██████████| 92/92 [01:51<00:00,  1.22s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:11<00:00,  1.06it/s]

                   all        365       2665      0.656      0.618      0.664      0.342



20 epochs completed in 0.705 hours.
Optimizer stripped from detect/train4/weights/last.pt, 136.7MB
Optimizer stripped from detect/train4/weights/best.pt, 136.7MB

Validating detect/train4/weights/best.pt...
Ultralytics YOLOv8.2.48 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68128383 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:16<00:00,  1.40s/it]


                   all        365       2665      0.655      0.618      0.664      0.342
         0: motorcycle        242        680      0.697      0.504      0.634       0.25
                1: car        293       1337      0.678      0.732       0.74      0.355
              2: truck         28         33      0.497      0.455      0.523      0.282
         3: pickup-van        125        159       0.66       0.61      0.602      0.331
                4: bus        256        456      0.744      0.789      0.821      0.494
Speed: 0.2ms preprocess, 26.2ms inference, 0.0ms loss, 9.5ms postprocess per image
Results saved to detect/train4


In [18]:
answer_list = []
# Use the model
model = YOLO('/kaggle/working/detect/train4/weights/best.pt')

for file in tqdm(glob(test_dir + '/*')):
    
    bbox_list = []
    cls_list = []
    scores_list = []

    # Predict on an image
    results = model(file, verbose=False)
    # Process results list
    for result in results:
        
        boxes = result.boxes
        for box in boxes:
            coor_box = box.xyxy.cpu().numpy().tolist()  # Boxes object for bounding box outputs
            class_box = box.cls.cpu().numpy().tolist()
            class_scores = box.conf.cpu().numpy().tolist()
        
            bbox_list.append(coor_box[0])
            cls_list.append(int(class_box[0]))
            scores_list.append(class_scores[0])
        
        value = (file.split('/')[-1], bbox_list, cls_list,scores_list )

    answer_list.append(value)

100%|██████████| 1552/1552 [01:50<00:00, 14.09it/s]


In [20]:
import pandas as pd
column_name = ['id','boxes', 'labels', 'scores']
xml_df = pd.DataFrame(answer_list, columns=column_name)
xml_df.to_csv('/kaggle/working/submission_yolo8x_20epoch.csv', index=None)